<a href="https://colab.research.google.com/github/MeenTers/Project/blob/main/Project_SI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
from torch import nn, load, utils
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from torchvision import datasets, transforms, models
from torchvision.models.vgg import model_urls
from os import path, listdir
model_urls['vgg19'] = model_urls['vgg19'].replace('https://', 'http://')

In [46]:
from scipy.spatial import distance_matrix
from scipy.spatial import distance

# Prepearing the model

In [4]:
class VGG:
	def __init__(self):
		model = models.vgg19(pretrained=True, progress=True)
		model.classifier = nn.Sequential(*list(model.classifier.children())[:3])
		self.model = model.cuda().eval()

	def __call__(self, x):
		return self.model(x)

In [5]:
vgg = VGG()

Downloading: "http://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

# Features Extraction

In [6]:
def get_features(model, loader):
    features = []
    with torch.no_grad():
        for batch, _ in tqdm(loader):
            if torch.cuda.is_available():
                batch = batch.cuda()
            b_features = model(batch).detach().cpu().numpy()
            for f in b_features:
                features.append(f)

    return features

# Loading the data

In [7]:
def get_dataset(images_path):
  transform = transforms.Compose([
    transforms.Resize(size=32),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
  ])

  dataset = datasets.ImageFolder(images_path, transform=transform)
  loader = utils.data.DataLoader(dataset, batch_size=100, shuffle=False, num_workers=1, pin_memory=True)
  return dataset, loader

# Test multi-view image

In [12]:
car_dataset , car_loader = get_dataset(r'/content/drive/MyDrive/Project_si/car_model/')

In [20]:
damage_dataset , damage_loader = get_dataset(r'/content/drive/MyDrive/Project_si/model_damage/')

In [17]:
car_base = get_features(model=vgg,loader=car_loader)

100%|██████████| 1/1 [00:00<00:00,  5.75it/s]


In [18]:
len(car_base[0])

4096

In [42]:
car_base[0]

array([0.      , 0.      , 3.960284, ..., 0.      , 0.      , 0.      ],
      dtype=float32)

In [21]:
car_damage = get_features(model=vgg,loader=damage_loader)

100%|██████████| 1/1 [00:01<00:00,  1.06s/it]


In [23]:
len(car_damage[0])

4096

# Distance

In [27]:
car_image = '/content/drive/MyDrive/Project_si/car_model/ID1'
damage_image = '/content/drive/MyDrive/Project_si/model_damage/ID1'

In [52]:
distance.euclidean(car_base[1], car_damage[1])

13.258048057556152